# Test of glm-pca on zebrafish data

This tutorial uses data from [Saunders, et al (2019)](https://elifesciences.org/articles/45181). Special thanks also go to [Lauren](https://twitter.com/LSaund11) for the tutorial improvement. 

In this [study](https://elifesciences.org/articles/45181), the authors profiled thousands of neural crest-derived cells from trunks of post-embryonic zebrafish. These cell classes include pigment cells, multipotent pigment cell progenitors, peripheral neurons, Schwann cells, chromaffin cells and others. These cells were collected during an active period of post-embryonic development, which has many similarities to fetal and neonatal development in mammals, when many of these cell types are migrating and differentiating as the animal transitions into its adult form. This study also explores the role of thyroid hormone (TH), a common endocrine factor, on the development of these different cell types. 

Such developmental and other dynamical processes are especially suitable for dynamo analysis as dynamo is designed to accurately estimate direction and magnitude of expression dynamics (`RNA velocity`), predict the entire lineage trajectory of any intial cell state (`vector field`), characterize the structure (`vector field topology`) of full gene expression space, as well as fate commitment potential (`single cell potential`). 

Import the package and silence some warning information (mostly `is_categorical_dtype` warning from anndata)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import dynamo as dyn 
from dynamo.configuration import DKM
import numpy as np

this is like R's sessionInfo() which helps you to debug version related bugs if any. 

In [2]:
dyn.get_all_dependencies_version()

package,dynamo-release,pre-commit,colorcet,cvxopt,hdbscan,loompy,matplotlib,networkx,numba,numdifftools,numpy,pandas,pynndescent,python-igraph,scikit-learn,scipy,seaborn,setuptools,statsmodels,tqdm,trimap,umap-learn
version,1.0.0,2.15.0,2.0.6,1.2.7,0.8.27,3.0.6,3.4.3,2.6.3,0.54.0,0.9.40,1.20.3,1.3.3,0.5.4,0.9.6,0.24.2,1.7.1,0.11.2,58.0.4,0.12.2,4.62.3,1.0.15,0.5.1


## Load data 

Dynamo comes with a few builtin sample datasets so you can familiarize with dynamo before analyzing your own dataset.
You can read your own data via `read`, `read_loom`, `read_h5ad`, `read_h5` (powered by the [anndata](https://anndata.readthedocs.io/en/latest/anndata.AnnData.html) package) or load_NASC_seq, etc. Here I just load the zebrafish sample data that comes with dynamo. This dataset has 4181 cells and 16940 genes. Its `.obs` attribute also included `condition`, `batch` information from the original study (you should also store those information to your `.obs` attribute which is essentially a Pandas Dataframe, see more at [anndata](https://anndata.readthedocs.io/en/latest/)). `Cluster`, `Cell_type`, umap coordinates that was originally analyzed with [Monocle 3](https://cole-trapnell-lab.github.io/monocle3/) are also provided. 

In [3]:
adata = dyn.sample_data.zebrafish()


|-----> Downloading data to ./data/zebrafish.h5ad


In [4]:
import scipy.sparse
from scipy import sparse
import anndata
def remove_rare_genes(counts,genes,minimum_detected_cells_per_gene):
    
    if type(counts) in [sparse.csr.csr_matrix, sparse.csc.csc_matrix]:
        
        #remove zero genes
        nonzero_genes_idx = np.array(counts.sum(axis=0)).flatten() > 0

        counts = counts[:,nonzero_genes_idx]
        genes = genes[nonzero_genes_idx]

        #count nonzero entries per gene
        nonzero_coords = counts.nonzero()
        n_nonzero = counts.count_nonzero()
        is_nonzero = sparse.csc_matrix((np.ones(n_nonzero),nonzero_coords))
        detected_cells_per_gene = np.array(is_nonzero.sum(axis=0)).flatten()

        keep_genes = detected_cells_per_gene >= minimum_detected_cells_per_gene    
        counts_kept = counts[:,keep_genes]
        genes_kept = genes[keep_genes]

        print('Of',len(detected_cells_per_gene),'total genes, returning',sum(keep_genes),'genes that are detected in %u or more cells.' % (minimum_detected_cells_per_gene))
        print('Output shape:', counts_kept.shape)

        return counts_kept,np.array(genes_kept)
    
    else:
        
        #remove zero genes
        nonzero_genes_idx = np.sum(counts,axis=0) > 0
        counts = counts[:,nonzero_genes_idx]
        genes = genes[nonzero_genes_idx]        
        
        #remove genes that are detected in less then n cells
        nonzero = counts > 0
        cells_per_gene = np.sum(nonzero,axis=0)
        include_genes = cells_per_gene >= minimum_detected_cells_per_gene
        counts_kept = counts[:,include_genes]
        genes_kept = genes[include_genes]
        print('Of',len(cells_per_gene),'total genes, returning',sum(include_genes),'genes that are detected in %u or more cells.' % (minimum_detected_cells_per_gene))
        print('Output shape:', counts_kept.shape)
        return counts_kept,genes_kept

def preprocess_adata_pearson_paper(adata):
    counts, genes, cells = adata.X, np.array(adata.var_names), adata.obs_names

    counts = counts.toarray()
    
    #remove low depth cells
    depths = np.array(np.sum(counts,axis=1)).flatten()
    minimum_depth = 500
    cells = cells[depths>minimum_depth]
    counts = counts[depths>minimum_depth,:]
    print('Of',len(depths),'cells, returning',sum(depths>minimum_depth),'cells that have a depth larger than', minimum_depth)
    print('New shape:', counts.shape)    
    
    counts, genes = remove_rare_genes(counts,genes,minimum_detected_cells_per_gene=5)
    counts = sparse.csr_matrix(counts)
    res = adata[np.unique(cells), np.unique(genes)]
    return res
    

In [5]:
import statsmodels.api as sm
from datetime import datetime
def compute_marginals(counts):
    '''compute depths per cell (ns) and relative expression fractions per gene (ps)'''
    ns = np.sum(counts,axis=1)
    ps = np.sum(counts,axis=0)
    ps = ps / np.sum(ps)    
    return np.squeeze(np.array(ns)), np.squeeze(np.array(ps))

def monitor_progress(gene_id,n_genes,print_time_every=1000,print_dot_every=25):
    if np.mod(gene_id,print_time_every)==0:
        print('')
        print('##',datetime.now().time(),'##',gene_id,'of',n_genes,'genes fit',end='')
    if np.mod(gene_id,print_dot_every)==0:
        print('.',end='')

def fit_offsetmodel_w_statsmodel(counts,depths,name):
    '''use statsmodel to fit offsetmodel (Eq. 3) and obtain beta0 (intercept) estimates, saving results to file'''
    n_cells = counts.shape[0]
    n_genes = counts.shape[1]    
    
    ##np.ones: will fit intercept beta0
    X = np.ones((n_cells,1))
    ##log(depths): will be used as offsets
    logdepths = np.log(depths)

    beta0 = np.zeros(n_genes) * np.nan
    for gene_id in range(n_genes):        
        offsetmodel = sm.Poisson(counts[:,gene_id], X, offset = logdepths)
        result = offsetmodel.fit(disp=0)
        beta0[gene_id] = result.params
        
        monitor_progress(gene_id,n_genes)


    res = dict(beta0=beta0)
    # np.save('fit_results/fit_offsetmodel_w_statsmodel_%s' % (name),res)


In [6]:
adata = preprocess_adata_pearson_paper(adata)

Of 4181 cells, returning 4157 cells that have a depth larger than 500
New shape: (4157, 16940)
Of 16589 total genes, returning 14555 genes that are detected in 5 or more cells.
Output shape: (4157, 14555)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
import glmpca
count_mat_T_array = adata.X.T.toarray()
print("np array conversion complete")
result = glmpca.glmpca.glmpca_customized(count_mat_T_array[:, :1000], 50, verbose=True)

np array conversion complete
running customized glm-pca ver...
Iteration: 0 | deviance=3.2436E+6
Iteration: 1 | deviance=3.2436E+6
Iteration: 2 | deviance=2.7742E+6
Iteration: 3 | deviance=2.4900E+6
Iteration: 4 | deviance=1.4977E+8
Iteration: 5 | deviance=1.7055E+80


GlmpcaError: Numerical divergence (deviance no longer finite), try increasing the penalty to improve stability of optimization.